In [4]:
import networkx as nx

def smallest_defensive_alliance(G):
    if not nx.is_connected(G):
        return "No defensive alliance exists for the input graph."

    # Initialize a list to store the vertices in the defensive alliance
    defensive_alliance = []

    # Iterate through the vertices of the graph
    for node in G.nodes():
        # Check if adding the current node to the defensive alliance is safe
        if is_safe(defensive_alliance + [node], G):
            defensive_alliance.append(node)

    return defensive_alliance

def is_safe(alliance, G):
    for u in alliance:
        for v in G.neighbors(u):
            # Check if there is an edge between u and v
            if G.has_edge(u, v):
                # Check if the edge is negative (conflict)
                if G[u][v]['sign'] == -1 and v not in alliance:
                    return False
            else:
                # Check if adding v to the alliance would create more than 3 neighbors
                if len(set(G.neighbors(v)).intersection(alliance)) > 3:
                    return False
    return True

# Example usage:
G = nx.Graph()
G.add_edges_from([(1, 2, {'sign': 1}), (1, 3, {'sign': 1}), (2, 3, {'sign': -1}), (3, 4, {'sign': 1})])

result = smallest_defensive_alliance(G)
print("Smallest defensive alliance:", result)


Smallest defensive alliance: [1, 4]
